## In questo file andiamo ad individuare le 5 feature più importanti per fare clustering

In [1]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join
import numpy as np
import os

In [5]:
path_file = os.path.abspath(os.getcwd()) #prendiamo il path in cui si trova il file su cui stiamo lavorando
print(path_file)
os.chdir(path_file) #cambiamo directory al fine di poter prendere i file csv per l'analisi della selezione delle features
data = r"\top_feature"

comb = list(combinations(range(0,5), 2))

d:\ProgettoAdvanced\AdvancedCyberSecurity


In [6]:
final_path = path_file + data
print(final_path)

d:\ProgettoAdvanced\AdvancedCyberSecurity\top_feature


In [19]:
anova = pd.DataFrame() #creazione di un dataframe vuoto
extra_tree = pd.DataFrame() #creazione di un dataframe vuoto
rfe = pd.DataFrame() #creazione di un dataframe vuoto
svm = pd.DataFrame() #creazione di un dataframe vuoto

In [22]:

for cart_corrente, sottocartelle, files in os.walk(final_path):
    for cartella in sottocartelle:
        results = os.path.join(final_path,cartella)
        for cartella_corrente, sottocart, file in os.walk(results):
            for csv_file in file:
                file_corrente = os.path.join(results,csv_file)
                df_csv = pd.read_csv(file_corrente)
                if file_corrente.__contains__("anova.csv"):
                    anova = pd.concat([anova,df_csv],ignore_index=True)
                elif file_corrente.__contains__("ExtraTree.csv"):
                    extra_tree = pd.concat([extra_tree,df_csv],ignore_index=True)
                elif file_corrente.__contains__("RFE.csv"):
                    rfe = pd.concat([rfe,df_csv],ignore_index=True)
                elif file_corrente.__contains__("SVM.csv"):
                    svm = pd.concat([svm,df_csv],ignore_index=True)


In [24]:
# Andiamo a normalizzare tutti gli score delle feature, in maniera tale da avere valori compresi tra 0 e 1

anova['F_Score'] = anova['F_Score']/anova['F_Score'].max()
extra_tree['F_Score'] = extra_tree['F_Score']/extra_tree['F_Score'].max()
svm['F_Score'] = svm['F_Score']/svm['F_Score'].max()
rfe['F_Score'] = rfe['F_Score']/rfe['F_Score'].max()

In [25]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,extra_tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,extra_tree,svm,rfe],ignore_index=True)

In [26]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

       Input_Features  Conto
16        Flow Pkts/s      6
25         Fwd Pkts/s      5
29  Init Bwd Win Byts      5
13      Flow IAT Mean      4
30  Init Fwd Win Byts      4
2     Bwd Pkt Len Max      3
3    Bwd Pkt Len Mean      3
8        ECE Flag Cnt      3
19       Fwd IAT Mean      3
32        Pkt Len Max      3
15       Flow IAT Std      2
24   Fwd Pkt Len Mean      2
21        Fwd IAT Std      2
1         Bwd IAT Tot      2
0        ACK Flag Cnt      2
14       Flow IAT Min      2
27   Fwd Seg Size Min      2
12       Flow IAT Max      2
11      Flow Duration      2
10        Flow Byts/s      2
34       Pkt Size Avg      2
6    Bwd Seg Size Avg      2
5          Bwd Pkts/s      2
35       RST Flag Cnt      2
31       PSH Flag Cnt      1
33       Pkt Len Mean      1
28      Fwd URG Flags      1
18        Fwd IAT Max      1
26   Fwd Seg Size Avg      1
23    Fwd Pkt Len Max      1
22        Fwd IAT Tot      1
20        Fwd IAT Min      1
17     Fwd Header Len      1
9        FIN F

In [27]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

       Input_Features  Conto
16        Flow Pkts/s      6
25         Fwd Pkts/s      5
29  Init Bwd Win Byts      5
13      Flow IAT Mean      4
30  Init Fwd Win Byts      4
2     Bwd Pkt Len Max      3
3    Bwd Pkt Len Mean      3
8        ECE Flag Cnt      3
19       Fwd IAT Mean      3
32        Pkt Len Max      3


In [28]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

       Input_Features   F_Score
11        Flow Pkts/s  0.005158
25        Flow Pkts/s  0.417705
31        Flow Pkts/s  0.514103
44        Flow Pkts/s  0.843071
66        Flow Pkts/s  1.000000
75        Flow Pkts/s  1.000000
13         Fwd Pkts/s  0.001804
24         Fwd Pkts/s  0.435594
32         Fwd Pkts/s  0.493936
45         Fwd Pkts/s  0.827847
65         Fwd Pkts/s  1.000000
7   Init Bwd Win Byts  0.640212
12  Init Bwd Win Byts  0.002768
43  Init Bwd Win Byts  0.914249
50  Init Bwd Win Byts  0.698144
60  Init Bwd Win Byts  1.000000
38      Flow IAT Mean  0.370623
55      Flow IAT Mean  0.338925
64      Flow IAT Mean  1.000000
76      Flow IAT Mean  1.000000
16  Init Fwd Win Byts  0.001626
23  Init Fwd Win Byts  0.486912
33  Init Fwd Win Byts  0.468611
61  Init Fwd Win Byts  1.000000
0     Bwd Pkt Len Max  1.000000
27    Bwd Pkt Len Max  0.340459
52    Bwd Pkt Len Max  0.448072
3    Bwd Pkt Len Mean  0.776309
57   Bwd Pkt Len Mean  0.335222
67   Bwd Pkt Len Mean  1.000000
2       

In [29]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [30]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
4        Flow Pkts/s  3.780038
7  Init Bwd Win Byts  3.255373
6         Fwd Pkts/s  2.759181
3      Flow IAT Mean  2.709548
2       ECE Flag Cnt  2.365106
1   Bwd Pkt Len Mean  2.111530
5       Fwd IAT Mean  1.997686
8  Init Fwd Win Byts  1.957148
0    Bwd Pkt Len Max  1.788531
9        Pkt Len Max  1.375986


In [31]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [32]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

      Input_Features    F_Score
0        Flow Pkts/s  22.680225
2  Init Bwd Win Byts  16.276866
1         Fwd Pkts/s  13.795903
3      Flow IAT Mean  10.838193
4  Init Fwd Win Byts   7.828592
7       ECE Flag Cnt   7.095318
6   Bwd Pkt Len Mean   6.334591
8       Fwd IAT Mean   5.993057
5    Bwd Pkt Len Max   5.365593
9        Pkt Len Max   4.127959


In [33]:
# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    
# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:

    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }

In [34]:
print(esito_finale)

           Feature_1          Feature_2
0        Flow Pkts/s  Init Bwd Win Byts
1        Flow Pkts/s         Fwd Pkts/s
2        Flow Pkts/s      Flow IAT Mean
3        Flow Pkts/s  Init Fwd Win Byts
4  Init Bwd Win Byts         Fwd Pkts/s
5  Init Bwd Win Byts      Flow IAT Mean
6  Init Bwd Win Byts  Init Fwd Win Byts
7         Fwd Pkts/s      Flow IAT Mean
8         Fwd Pkts/s  Init Fwd Win Byts
9      Flow IAT Mean  Init Fwd Win Byts


In [ ]:
top5 = esito_finale.Feature_1.unique()
top5 = np.append(top5,esito_finale.Feature_2.unique())
top5 = pd.Series(top5)
top5 = top5.unique()
top5 = pd.Series(top5)
print(top5)

top5.to_csv('top_feature/top5_totale.csv',index=False)

In [35]:
esito_finale.to_csv('top_feature/combinazioni_totale.csv',index=False,header=False)